## The analysis covered speed violations for different vehicle categories depending on the time of day.

### Input data:

* Maximum speed limit for passenger cars: 90 km/h
* Maximum speed limit for trucks: 70 km/h
* Cat_1: passenger cars and passenger cars with trailers
* Cat_2: 2- and 3-axle trucks
* Cat_3: lorries with trailers
* Cat_4: truck tractors with trailers
* Cat_5: buses
* Cat_6: non-classified vehicles

In [1]:
!gdown --id 1gLTGu9JzvJKZd_l9q_gQQ7M_FKIZYHXn

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1gLTGu9JzvJKZd_l9q_gQQ7M_FKIZYHXn
To: /content/PZZTD_dane źródłowe_1.xlsx
100% 71.9M/71.9M [00:00<00:00, 180MB/s]


In [2]:
from datetime import datetime

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_excel('/content/PZZTD_dane źródłowe_1.xlsx')

In [3]:
df.insert(
    loc=9, 
    column="timestamp", 
    value=df.eventtimestamp.values.astype(np.int64) // 10**9)

df = df.drop(
        labels=['godzina', 'data', 'Godzina zamieniona na sekundy', 
                'różnica w naciskach', 'Unnamed: 32', 'Unnamed: 33', 
                'Unnamed: 34', 'Unnamed: 35'], 
        axis=1)

df.columns = ['record_id', 'speed', 'axlenum', 'grossweight', 
              'leftwheelspressures', 'rightwheelspressures', 'axlespressures', 
              'vehiclelength', 'eventtimestamp', 'timestamp', 
              'time_difference_between_veihcles','category8_1', 'axlesdistances', 
              'cost323', 'leftwheelspressures_1', 'leftwheelspressures_2', 
              'leftwheelspressures_3', 'leftwheelspressures_4', 
              'leftwheelspressures_5', 'rightwheelspressures_1', 
              'rightwheelspressures_2', 'rightwheelspressures_3', 
              'rightwheelspressures_4', 'rightwheelspressures_5', 
              'axlespressures_1', 'axlespressures_2', 'axlespressures_3', 
              'axlespressures_4', 'axlespressures_5']

df.loc[df['cost323'] == 3, 'cost323'] = 2
df.loc[df['cost323'] == 5, 'cost323'] = 4
df.loc[df['cost323'] == 6, 'cost323'] = 3
df.loc[df['cost323'] == 7, 'cost323'] = 5
df.loc[df['cost323'] == 8, 'cost323'] = 6

df = df.set_index('eventtimestamp')
df.head(3)

,record_id,speed,axlenum,grossweight,leftwheelspressures,rightwheelspressures,axlespressures,vehiclelength,timestamp,time_difference_between_veihcles,...,rightwheelspressures_1,rightwheelspressures_2,rightwheelspressures_3,rightwheelspressures_4,rightwheelspressures_5,axlespressures_1,axlespressures_2,axlespressures_3,axlespressures_4,axlespressures_5
eventtimestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-20 00:00:29,1415933,53,2,1913,"{512,427}","{545,429}","{1057,856}",482,1639958429,NaN,...,545,429,NaN,NaN,NaN,1057,856,NaN,NaN,NaN
2021-12-20 00:00:30,1415935,53,2,1773,"{490,415}","{480,388}","{970,803}",517,1639958430,1.0,...,480,388,NaN,NaN,NaN,970,803,NaN,NaN,NaN
2021-12-20 00:03:37,1415937,66,2,1391,"{354,347}","{328,362}","{682,709}",481,1639958617,187.0,...,328,362,NaN,NaN,NaN,682,709,NaN,NaN,NaN


### **Sunrises and sunsets**
* 20.12.2021: 7:44 - 15:28
* 27.12.2021: 7:46 - 15:28
* 03.01.2022: 7:45 - 15:36
* 10.01.2022: 7:42 - 15:45
* 17.01.2022: 7:35 - 15:57
* 24.01.2022: 7:27 - 16:09
* 31.01.2022: 7:17 - 16:22

In [23]:
week_1 = df.loc[:'2021-12-26']
week_2 = df.loc['2021-12-27':'2022-01-02']
week_3 = df.loc['2022-01-03':'2022-01-09']
week_4 = df.loc['2022-01-10':'2022-01-16']
week_5 = df.loc['2022-01-17':'2022-01-23']
week_6 = df.loc['2022-01-24':'2022-01-30']
week_7 = df.loc['2022-01-31':]

In [24]:
day_week_1 = week_1.between_time(start_time='7:44', end_time='15:28', 
                                 include_start=True, include_end=False)
day_week_2 = week_2.between_time(start_time='7:46', end_time='15:28', 
                                 include_start=True, include_end=False)
day_week_3 = week_3.between_time(start_time='7:45', end_time='15:36', 
                                 include_start=True, include_end=False)
day_week_4 = week_4.between_time(start_time='7:42', end_time='15:45', 
                                 include_start=True, include_end=False)
day_week_5 = week_5.between_time(start_time='7:35', end_time='15:57', 
                                 include_start=True, include_end=False)
day_week_6 = week_6.between_time(start_time='7:27', end_time='16:09', 
                                 include_start=True, include_end=False)
day_week_7 = week_7.between_time(start_time='7:17', end_time='16:22', 
                                 include_start=True, include_end=False)

days = pd.concat([day_week_1, day_week_2, day_week_3, 
                  day_week_4, day_week_5, day_week_6, 
                  day_week_7])

In [25]:
night_week_1 = week_1.between_time(end_time='7:44', start_time='15:28', 
                                   include_start=True, include_end=False)
night_week_2 = week_2.between_time(end_time='7:46', start_time='15:28', 
                                   include_start=True, include_end=False)
night_week_3 = week_3.between_time(end_time='7:45', start_time='15:36', 
                                   include_start=True, include_end=False)
night_week_4 = week_4.between_time(end_time='7:42', start_time='15:45', 
                                   include_start=True, include_end=False)
night_week_5 = week_5.between_time(end_time='7:35', start_time='15:57', 
                                   include_start=True, include_end=False)
night_week_6 = week_6.between_time(end_time='7:27', start_time='16:09', 
                                   include_start=True, include_end=False)
night_week_7 = week_7.between_time(end_time='7:17', start_time='16:22', 
                                   include_start=True, include_end=False)

nights = pd.concat([night_week_1, night_week_2, night_week_3, night_week_4,
                    night_week_5, night_week_6, night_week_7])

In [26]:
passenger_cars_speeds = [v for v in range(90, 121, 5)]
truck_speeds = [v for v in range(70, 101, 5)]

# Functions

In [27]:
def get_number_of_overspeeding(
    df: pd.DataFrame,
    list_of_infringements: list,
    vehicle_speed: list,
    vehicle_categories: list):
  
  for category in vehicle_categories:
    for i, _ in enumerate(vehicle_speed):
      if (i == 0):
        list_of_infringements.append(df[(df['cost323'] == category) & 
                                        (df['speed'] <= vehicle_speed[i])
                                       ]
                                    )
      elif (i == len(vehicle_speed) - 1):
        list_of_infringements.append(df[(df['cost323'] == category) & 
                                        (df['speed'] <= vehicle_speed[i]) & 
                                        (df['speed'] > vehicle_speed[i - 1])
                                       ]
                                    )
        list_of_infringements.append(df[(df['cost323'] == category) & 
                                        (df['speed'] > vehicle_speed[i])
                                       ]
                                    )
      else:
        list_of_infringements.append(df[(df['cost323'] == category) & 
                                        (df['speed'] <= vehicle_speed[i]) & 
                                        (df['speed'] > vehicle_speed[i - 1])
                                       ]
                                    )

In [28]:
def premare_dataframe_of_overspeeding(
    time_of_day_for_passenger_cars: list,
    time_of_day_for_trucks: list):

  trucks_overspeeding = np.array([time.shape[0] 
                                  for time in time_of_day_for_trucks]
                                )
  trucks_overspeeding = np.reshape(a=trucks_overspeeding, 
                                   newshape=(-1, (len(truck_speeds) + 1))
                                  )
  vehicle_table = pd.DataFrame(data=trucks_overspeeding, 
                               index=['Cat_2', 'Cat_3', 
                                      'Cat_4', 'Cat_5', 'Cat_6']
                              )

  vehicle_table = vehicle_table.T
  vehicle_table.insert(loc=0, 
                       column='Cat_1', 
                       value=[time.shape[0]
                              for time in time_of_day_for_passenger_cars])
  list_of_overspeeding = ['correct_speed']
  for speed in truck_speeds[1:]:
    v = speed - truck_speeds[0]
    list_of_overspeeding.append(f"exceeding_by_{v}_km_per_hour")

  list_of_overspeeding.append(f"exceeding_by_more_than_{v}_km_per_hour")

  vehicle_table.index = list_of_overspeeding
  return vehicle_table

In [29]:
def prepare_data_for_histogram(vehicle_table):
  number_of_cars = [vehicle_table[col_name].sum() 
                    for col_name in vehicle_table.columns]
  return (vehicle_table / number_of_cars) * 100

In [44]:
def create_hist_subplots(number_of_cars: pd.DataFrame, time_of_day: str):
  fig = make_subplots(rows=2, cols=3, shared_yaxes=True,
                      subplot_titles=['Category 1', 'Category 2',
                                      'Category 3', 'Category 4',
                                      'Category 5', 'Category 6'])
  
  fig.add_trace(go.Bar(y=number_of_cars['Cat_1']), row=1, col=1)
  fig.add_trace(go.Bar(y=number_of_cars['Cat_2']), row=1, col=2)
  fig.add_trace(go.Bar(y=number_of_cars['Cat_3']), row=1, col=3)
  fig.add_trace(go.Bar(y=number_of_cars['Cat_4']), row=2, col=1)
  fig.add_trace(go.Bar(y=number_of_cars['Cat_5']), row=2, col=2)
  fig.add_trace(go.Bar(y=number_of_cars['Cat_6']), row=2, col=3)

  for row in range(1, 3):
    for col in range(1, 4):
      fig.update_xaxes(title_text='OVERSPEEDING [km/h]', row=row, col=col)
      fig.update_yaxes(title_text='PERCENTAGE [%]', row=row, col=col)

  fig.update_layout(title="PERCENTAGE DISTRIBUTION OF SPEEDING VIOLATIONS " 
                    f"FOR EACH CATEGORY - {time_of_day.upper()}", 
                    showlegend=False)
  fig.show()

# General division

In [31]:
list_of_passenger_cars_overspeeding_general = []
list_of_trucks_overspeeding_general = []

get_number_of_overspeeding(
    df=df, 
    list_of_infringements=list_of_passenger_cars_overspeeding_general, 
    vehicle_speed=passenger_cars_speeds, 
    vehicle_categories=[1])

get_number_of_overspeeding(
    df=df, 
    list_of_infringements=list_of_trucks_overspeeding_general, 
    vehicle_speed=truck_speeds, 
    vehicle_categories=[category for category in range(2, 7)])

In [32]:
df_number_of_cars = premare_dataframe_of_overspeeding(
    list_of_passenger_cars_overspeeding_general, 
    list_of_trucks_overspeeding_general)
df_number_of_cars

,Cat_1,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6
correct_speed,270516,7083,2568,18515,3485,13
exceeding_by_5_km_per_hour,565,211,53,521,35,0
exceeding_by_10_km_per_hour,248,68,22,212,7,0
exceeding_by_15_km_per_hour,145,19,3,83,5,0
exceeding_by_20_km_per_hour,62,6,4,49,0,1
exceeding_by_25_km_per_hour,34,2,0,1,0,0
exceeding_by_30_km_per_hour,16,0,0,0,0,0
exceeding_by_more_than_30_km_per_hour,34,0,0,0,0,0


In [33]:
histogram_data = prepare_data_for_histogram(df_number_of_cars)[1:]
histogram_data

,Cat_1,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6
exceeding_by_5_km_per_hour,0.208011,2.855596,2.000000,2.688200,0.990940,0.000000
exceeding_by_10_km_per_hour,0.091304,0.920287,0.830189,1.093855,0.198188,0.000000
exceeding_by_15_km_per_hour,0.053383,0.257139,0.113208,0.428254,0.141563,0.000000
exceeding_by_20_km_per_hour,0.022826,0.081202,0.150943,0.252825,0.000000,7.142857
exceeding_by_25_km_per_hour,0.012517,0.027067,0.000000,0.005160,0.000000,0.000000
exceeding_by_30_km_per_hour,0.005891,0.000000,0.000000,0.000000,0.000000,0.000000
exceeding_by_more_than_30_km_per_hour,0.012517,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
create_hist_subplots(histogram_data, 'general')

# Division by days


In [35]:
list_of_passenger_cars_overspeeding_day = []
list_of_trucks_overspeeding_day = []

get_number_of_overspeeding(
    df=days, 
    list_of_infringements=list_of_passenger_cars_overspeeding_day, 
    vehicle_speed=passenger_cars_speeds, 
    vehicle_categories=[1])

get_number_of_overspeeding(
    df=days, 
    list_of_infringements=list_of_trucks_overspeeding_day, 
    vehicle_speed=truck_speeds, 
    vehicle_categories=[category for category in range(2, 7)])

In [36]:
df_number_of_cars_day = premare_dataframe_of_overspeeding(
    list_of_passenger_cars_overspeeding_day, 
    list_of_trucks_overspeeding_day)
df_number_of_cars_day

,Cat_1,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6
correct_speed,143741,4798,1520,10396,1467,4
exceeding_by_5_km_per_hour,187,102,17,102,14,0
exceeding_by_10_km_per_hour,82,25,3,22,3,0
exceeding_by_15_km_per_hour,39,8,0,5,2,0
exceeding_by_20_km_per_hour,21,2,0,0,0,1
exceeding_by_25_km_per_hour,12,0,0,0,0,0
exceeding_by_30_km_per_hour,3,0,0,0,0,0
exceeding_by_more_than_30_km_per_hour,8,0,0,0,0,0


In [37]:
histogram_data_day = prepare_data_for_histogram(df_number_of_cars_day)[1:]
histogram_data_day

,Cat_1,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6
exceeding_by_5_km_per_hour,0.129777,2.066869,1.103896,0.969121,0.942127,0.0
exceeding_by_10_km_per_hour,0.056908,0.506586,0.194805,0.209026,0.201884,0.0
exceeding_by_15_km_per_hour,0.027066,0.162107,0.000000,0.047506,0.134590,0.0
exceeding_by_20_km_per_hour,0.014574,0.040527,0.000000,0.000000,0.000000,20.0
exceeding_by_25_km_per_hour,0.008328,0.000000,0.000000,0.000000,0.000000,0.0
exceeding_by_30_km_per_hour,0.002082,0.000000,0.000000,0.000000,0.000000,0.0
exceeding_by_more_than_30_km_per_hour,0.005552,0.000000,0.000000,0.000000,0.000000,0.0


In [38]:
create_hist_subplots(histogram_data_day, 'day')

# Division by night

In [39]:
list_of_passenger_cars_overspeeding_night = []
list_of_trucks_overspeeding_night = []

get_number_of_overspeeding(
    df=nights, 
    list_of_infringements=list_of_passenger_cars_overspeeding_night, 
    vehicle_speed=passenger_cars_speeds, 
    vehicle_categories=[1])

get_number_of_overspeeding(
    df=nights, 
    list_of_infringements=list_of_trucks_overspeeding_night, 
    vehicle_speed=truck_speeds, 
    vehicle_categories=[category for category in range(2, 7)])

In [40]:
df_number_of_cars_night = premare_dataframe_of_overspeeding(
    list_of_passenger_cars_overspeeding_night, 
    list_of_trucks_overspeeding_night)
df_number_of_cars_night

,Cat_1,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6
correct_speed,126775,2285,1048,8119,2018,9
exceeding_by_5_km_per_hour,378,109,36,419,21,0
exceeding_by_10_km_per_hour,166,43,19,190,4,0
exceeding_by_15_km_per_hour,106,11,3,78,3,0
exceeding_by_20_km_per_hour,41,4,4,49,0,0
exceeding_by_25_km_per_hour,22,2,0,1,0,0
exceeding_by_30_km_per_hour,13,0,0,0,0,0
exceeding_by_more_than_30_km_per_hour,26,0,0,0,0,0


In [41]:
histogram_data_night = prepare_data_for_histogram(df_number_of_cars_night)[1:]
histogram_data_night

,Cat_1,Cat_2,Cat_3,Cat_4,Cat_5,Cat_6
exceeding_by_5_km_per_hour,0.296408,4.441728,3.243243,4.731256,1.026393,0.0
exceeding_by_10_km_per_hour,0.130169,1.752241,1.711712,2.145438,0.195503,0.0
exceeding_by_15_km_per_hour,0.083120,0.448248,0.270270,0.880759,0.146628,0.0
exceeding_by_20_km_per_hour,0.032150,0.162999,0.360360,0.553297,0.000000,0.0
exceeding_by_25_km_per_hour,0.017251,0.081500,0.000000,0.011292,0.000000,0.0
exceeding_by_30_km_per_hour,0.010194,0.000000,0.000000,0.000000,0.000000,0.0
exceeding_by_more_than_30_km_per_hour,0.020388,0.000000,0.000000,0.000000,0.000000,0.0


In [42]:
create_hist_subplots(histogram_data_night, 'night')